In [14]:
from data_generation.well_generator import FakeWell

ModuleNotFoundError: No module named 'tests.data_generation'

All FakeWell instances are held within a FakeRun instance that assigns barcodes to the fake wells, handles fake input generation, and builds expected outputs.

In [12]:
# Class that holds information for a test run
class FakeRun():
    def __init__(self, detailed = True):
        """
        fakewells: A list of fully prepared FakeWell objects.
        detailed: Whether or not we are using a detailed refseq file. 
        """
        # Set a random configuration
        self.config = Config(detailed = detailed)
                
        # Build wells
        self.wells = [None] * len(INDEX_DF)
        unique_plates_found = {}
        refseq_counter = -1
        for i, row in enumerate(INDEX_DF.itertuples()):
            
            # One path for detailed, increment the refseq counter
            if detailed:
                refseq_counter += 1
            
            # Otherwise, only increment the counter if it is for a new
            # plate
            elif row.IndexPlate not in unique_plates_found:
                unique_plates_found.add(row.IndexPlate)
                refseq_counter += 1
        
            # Create a new well
            well = FakeWell(self.config,
                            self.config.refseqs[refseq_counter])
            
            # Assign information to the well
            well.platename = row.IndexPlate
            well.wellname = row.Well
            well.f_barcode = row.FBC
            well.r_barcode = row.RBC
            
            # Record
            self.wells[i] = well
            
    def build_input(self, saveloc):        
        
        warnings.warn("Add an NNN version of the refseq")
        
        # Loop over all wells. Generate reads
        forward_reads = []
        reverse_reads = []
        for well in self.wells:
            
            # Produce reads if this is not a dud
            if not well.dud_well:
                f_well_reads, r_well_reads = well.build_all_reads()
                forward_reads.extend(f_well_reads)
                reverse_reads.extend(r_well_reads)
        
        # Return the fastq files ready for processing
        f_savename = os.path.join(saveloc, f"testinput_R1_allreads.fastq")
        r_savename = os.path.join(saveloc, f"testinput_R2_allreads.fastq")
        with open(f_savename, "w") as f:
            f.write("\n".join(forward_reads))
        with open(r_savename, "w") as f:
            f.write("\n".join(reverse_reads))
            
        # Build reference sequence files
        
    def build_output_counts(self):
        """
        Builds output files for the different `OutputCounts`
        """
        pass

In [10]:
test_run = FakeRun()

/tmp/ipykernel_18162/616808641.py:30: UserWarning: You need to make sure that the amino acid returned is different from the existing one. Same goes for the nucleotides.
  warnings.warn("You need to make sure that the amino acid returned is different"
/tmp/ipykernel_18162/2844603749.py:35: UserWarning: Add `murder` to list of functions
  warnings.warn("Add `murder` to list of functions")


In [11]:
test_run.build_input("./")

To do:
1. Output refseq
2. Fix warnings
3. Run a test
4. Get output counts